# Perform some data selection and grouping

In [ ]:
import sys

pwd = !echo ${PWD}
sys.path.append(pwd[0]+"/../../../code/local/bin")

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy import ndimage
import math
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.ticker import FormatStrFormatter

import seppy
import os

datapath=pwd[0]+"/../dat/"
figpath=pwd[0]+"/../fig/"

In [ ]:
# read data, mask, headers, model, source functions

sep = seppy.sep()

data_axes, data = sep.read_file(datapath+"ch6_data.H")
pdata = data.reshape(data_axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch6_data.HH")
pattr = data.reshape(axes.n,order='F').T

axes, data = sep.read_file(datapath+"ch6_mute.H")
pmute = data.reshape(axes.n,order='F').T

source_axes, data = sep.read_file(datapath+"ch6_all_time_func.H")
psrc = data.reshape(source_axes.n,order='F').T

model_axes, data = sep.read_file(datapath+"ch6_model_scan.H")
pmodel = data.reshape(model_axes.n,order='F').T


dt=data_axes.d[0]
dx=data_axes.d[1]
nt=data_axes.n[0]
ntr=data_axes.n[1]
ns=data_axes.n[2]

sxyz=pattr[:,0,[2, 12, 13, 14, 0, 1]]
sxyz[:,1:4] *= 0.001 # convert to km

srxyz=pattr[:,:,[3, 15, 16, 17]]
srxyz[:,:,1:4] *= 0.001 # convert to km

In [ ]:
# apply constant time median filtering to the data to remove CMN
for s in range(ns):
    for it in range(pdata.shape[2]):
        med = np.median(pdata[s,:,it])
        pdata[s,:,it] -= med
pdata *= pmute

In [ ]:
extent=[model_axes.o[1],model_axes.o[1]+(model_axes.n[1]-1)*model_axes.d[1],model_axes.o[0]+(model_axes.n[0]-1)*model_axes.d[0],model_axes.o[0]]
# ymed = np.median(sxyz[:,2])
ymed=np.median(srxyz[:,:,2])
iy = int((ymed - model_axes.o[2])/model_axes.d[2])

plt.figure(figsize=(16,6))
plt.imshow(np.transpose(pmodel[0,iy,:,:]),cmap='jet',extent=extent,vmin=2.5,vmax=6.0,interpolation='bilinear', aspect='auto')
plt.colorbar().set_label(label="km/s",size=16)
plt.scatter(srxyz[:,:,1],srxyz[:,:,3],s=5,c='y',marker='.')
plt.scatter(sxyz[:,1],sxyz[:,3],s=50,c=sxyz[:,4],marker='x')
plt.xlabel(r'X (km)',fontsize=16)
plt.ylabel(r'Depth (km)',fontsize=16)

In [ ]:
# keep the last or second to last perf in each stage
select1=np.array([4,8,11,14,19,23,26,29,32,36,40])
select2=select1 - 1

sxyz1=sxyz[select1,:]
sxyz2=sxyz[select2,:]
srxyz1=srxyz[select1,:,:]
srxyz2=srxyz[select2,:,:]

In [ ]:
extent=[model_axes.o[1],model_axes.o[1]+(model_axes.n[1]-1)*model_axes.d[1],model_axes.o[0]+(model_axes.n[0]-1)*model_axes.d[0],model_axes.o[0]]
# ymed = np.median(sxyz[:,2])
ymed=np.median(srxyz[:,:,2])
iy = int((ymed - model_axes.o[2])/model_axes.d[2])

plt.figure(figsize=(16,6))
plt.imshow(np.transpose(pmodel[0,iy,:,:]),cmap='jet',extent=extent,vmin=2.5,vmax=6.0,interpolation='bilinear', aspect='auto')
plt.colorbar().set_label(label="km/s",size=16)
plt.scatter(sxyz1[:,1],sxyz1[:,3],s=50,c=sxyz1[:,4],marker='x')
plt.xlabel(r'X (km)',fontsize=16)
plt.ylabel(r'Depth (km)',fontsize=16)

In [ ]:
# select shots and create groups

# group 1: last perf in each stage
pdata1=pdata[select1,:,:]
pmute1=pmute[select1,:,:]
psrc1=psrc[select1,:,:]
pattr1=pattr[select1,:,:]

# group 2: second to last perf in each stage
pdata2=pdata[select2,:,:]
pmute2=pmute[select2,:,:]
psrc2=psrc[select2,:,:]
pattr2=pattr[select2,:,:]

In [ ]:
# save the selections (data, mask, sources, headers)
sep.write_file(datapath+"ch6_data0_group1.H", np.transpose(pdata1), ds=np.array([dt,1,1]), os=np.array([0,-600,0]), dpath=datapath)
sep.write_file(datapath+"ch6_data0_group2.H", np.transpose(pdata2), ds=np.array([dt,1,1]), os=np.array([0,-600,0]), dpath=datapath)
sep.write_file(datapath+"ch6_data0_group1.HH", np.transpose(pattr1), ds=np.array([1,1,1]), os=np.array([0,-600,0]), dpath=datapath)
sep.write_file(datapath+"ch6_data0_group2.HH", np.transpose(pattr2), ds=np.array([1,1,1]), os=np.array([0,-600,0]), dpath=datapath)
sep.write_file(datapath+"ch6_mute0_group1.H", np.transpose(pmute1), ds=np.array([dt,1,1]), os=np.array([0,-600,0]), dpath=datapath)
sep.write_file(datapath+"ch6_mute0_group2.H", np.transpose(pmute2), ds=np.array([dt,1,1]), os=np.array([0,-600,0]), dpath=datapath)
sep.write_file(datapath+"ch6_sources0_group1.H", np.transpose(psrc1), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)
sep.write_file(datapath+"ch6_sources0_group2.H", np.transpose(psrc2), ds=np.array([dt,1,1]), os=np.array([0,0,0]), dpath=datapath)